In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Deep Learning Project/output/224/cardamom_dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 1104 images belonging to 3 classes.


In [7]:
validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Deep Learning Project/output/224/cardamom_dataset/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 33 images belonging to 3 classes.


In [8]:
base_model = tf.keras.applications.ResNet152V2(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [11]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

print("Model compiled successfully!")

Model compiled successfully!


In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[es, mc]
)

# Load best model
model.load_weights('best_model.h5')

# Evaluate on test set
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Deep Learning Project/output/224/cardamom_dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

loss, accuracy, precision, recall = model.evaluate(test_generator, verbose=0)
f1_score = 2 * (precision * recall) / (precision + recall)

print('Test set performance:')
print('Loss:', loss)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1_score)

# Save model
model.save('my_model.h5')

Epoch 1/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 504ms/step - accuracy: 0.9794 - loss: 0.0586 - precision: 0.9794 - recall: 0.9794
Epoch 1: val_accuracy did not improve from 0.96875
34/34 ━━━━━━━━━━━━━━━━━━━━ 19s 557ms/step - accuracy: 0.9793 - loss: 0.0589 - precision: 0.9793 - recall: 0.9793 - val_accuracy: 0.9688 - val_loss: 0.2269 - val_precision: 0.9688 - val_recall: 0.9688
Epoch 2/20
 1/34 ━━━━━━━━━━━━━━━━━━━━ 6s 188ms/step - accuracy: 1.0000 - loss: 0.0124 - precision: 1.0000 - recall: 1.0000
Epoch 2: val_accuracy did not improve from 0.96875
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 1.0000 - loss: 0.0124 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.9062 - val_loss: 0.1267 - val_precision: 0.9355 - val_recall: 0.9062
Epoch 3/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 514ms/step - accuracy: 0.9674 - loss: 0.0705 - precision: 0.9695 - recall: 0.9674
Epoch 3: val_accuracy did not improve from 0.96875
34/34 ━━━━━━━━━━━━━━━━━━━━ 20s 585ms/step - accuracy: 0.9676 - loss: 0.0705 - pr

Test set performance:
Loss: 0.07318827509880066
Accuracy: 0.982758641242981
Precision: 0.982758641242981
Recall: 0.982758641242981
F1-score: 0.982758641242981
